In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Dataset, WeightedRandomSampler
import torch
import torch.nn.functional as F
import pandas as pd
import torchvision
from tqdm import tqdm
from torch.utils.data import DataLoader, Dataset, WeightedRandomSampler
from torchmetrics.classification import MulticlassConfusionMatrix, F1Score, MulticlassPrecision, MulticlassRecall, MulticlassPrecisionRecallCurve, MulticlassROC
from torch.optim import lr_scheduler
import torch.nn as nn
import torch
import pytorch_lightning as pl
from torch.utils.tensorboard import SummaryWriter
from matplotlib.backends.backend_agg import FigureCanvasAgg

import confinement_mode_classifier as cmc
import alt_models as am


In [2]:
signal_window = 320
num_workers = 6
signal_name = 'mc'
batch_size = 512
pl.seed_everything(42)
path=os.getcwd()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
shot_usage = pd.read_csv(f'{path}/data/shot_usage.csv')
shot_for_ris = shot_usage[shot_usage['used_for_alt']]
shot_numbers = shot_for_ris['shot']
shots_for_testing = shot_for_ris[shot_for_ris['used_as'] == 'test']['shot']
shots_for_validation = shot_for_ris[shot_for_ris['used_as'] == 'val']['shot']


Seed set to 42


In [3]:
from importlib import reload
reload(am)

<module 'alt_models' from '/compass/Shared/Users/bogdanov/vyzkumny_ukol/alt_models.py'>

In [3]:

shot_df, test_df, val_df, train_df = am.split_df(path, shot_numbers, shots_for_testing, shots_for_validation, use_ELMS=True, signal_name=signal_name)


# Create dataloaders
train_dataloader = am.get_dloader(train_df, batch_size=batch_size, 
                            balance_data=True, shuffle=False, 
                            signal_window=signal_window,
                            signal_name=signal_name,
                            num_workers=num_workers)

val_dataloader = am.get_dloader(val_df, batch_size=batch_size, 
                            balance_data=True, shuffle=False, 
                            signal_window=signal_window,
                            signal_name=signal_name,
                            num_workers=num_workers)

test_dataloader = am.get_dloader(test_df, batch_size=batch_size, 
                            balance_data=False, shuffle=False, 
                            signal_window=signal_window,
                            signal_name=signal_name,
                            num_workers=num_workers)

In [8]:
model = am.select_model_architecture('InceptionTime', num_classes=3, window=320)
model.load_state_dict(torch.load(f'/compass/Shared/Users/bogdanov/vyzkumny_ukol/runs/24-02-22, 18-31-57 InceptionTime_on_Mink_coil_320dpoints_16_epochs/model.pt'))
model.eval()

model.to(device)

torch.Size([512, 320, 4])

In [15]:
optimizer = torch.optim.AdamW(model.parameters(), lr=200, weight_decay=0.01)
exp_lr_scheduler = lr_scheduler.OneCycleLR(optimizer, max_lr=0.01, total_steps=24) #!!!
learning_rates = []
for i in range(24):
    a = model(next(iter(train_dataloader))[signal_name].to(device).float())
    optimizer.step()
    exp_lr_scheduler.step()
    learning_rates.append(exp_lr_scheduler.get_last_lr())